In [4]:
import subprocess
import pandas as pd
import os
os.environ['INSTANCE_NAME'] = 'LOOP_script'
from global_settings import simexSettings
script_dir = os.path.abspath('')
results_dir = os.path.join(script_dir, f'{simexSettings["results_dir"]}')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)

from DLASIUT_find_best_scenarios import automatic_performance
scripts = [ 'sumo_vsl_run.py', 'sumo_novsl_run.py']
files = []
# RUn scripts
for script in scripts:
    
    cmd = ['python3', script]
    lines = subprocess.run(cmd,stdout=subprocess.PIPE).stdout.splitlines()
    print(f" LINES: {lines}")
    for line in lines:
        output = "{}".format(line.rstrip().decode("utf-8"))
        if 'simex_output' in output:
            files.append(output)
        
    
# collect name of the csv file
df_baseline = pd.read_csv(files[1])
df_control = pd.read_csv(files[0])

# Main Function receives np.arrays
dataset_baseline = df_baseline.to_numpy()
dataset_control = df_control.to_numpy()

# Run martin script on the csv files
_incremnet_step_for_x = 10
_max_order_of_polynom = 9
_tolerance_in_diffrence=12
results = automatic_performance(dataset_baseline,dataset_control,incremnet_step_for_x=_incremnet_step_for_x,max_order_of_polynom=_max_order_of_polynom,tolerance_in_diffrence=_tolerance_in_diffrence)
print(results)


# get moddifiers ?? providing the initial x values to the script???
from modifier_controller import ModifierController
from components_configuration import components
print(f"Results {results}")
mod_outcome = ModifierController.control(intervals_list=results, selected_modifier=components['modifierA'],
                                             do_plot=simexSettings['do_plot'])
mod_outcome

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 205, in _process_chunk
    return [fn(*args) for args in chunk]
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 205, in <listcomp>
    return [fn(*args) for args in chunk]
  File "/home/amy/tmp/SimEx/notebooks/simulator.py", line 66, in sumo_simulator_vsl
    traci.start(sumoCmd)
  File "/home/amy/tmp/repos/SimEx/venv/lib/python3.10/site-packages/traci/main.py", line 145, in start
    sumoProcess = subprocess.Popen(cmd2, stdout=stdout)
  File "/usr/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr/lib/python3.10/subprocess.py", line 1863, in _execute_child
    raise child_exception_type(errno_num, 

 LINES: [b'Modifier...', b'[MODC]: (interval_min_tick, interval_max_tick):  (2500, 4000)', b'[MODC]: mod_ticks:  [2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900]', b'Temp x: [2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900]', b'[MODC]: mod_x:  [2600.0, 2681.5384615384614, 2766.153846153846, 2853.846153846154, 2944.6153846153848, 3038.4615384615386, 3135.3846153846152, 3235.3846153846152, 3338.4615384615386, 3444.6153846153848, 3553.846153846154, 3666.153846153846, 3781.5384615384614, 3900.0]', b'MAIN mod outcome ([[2600.0, 2681.5384615384614, 2766.153846153846, 2853.846153846154, 2944.6153846153848, 3038.4615384615386, 3135.3846153846152, 3235.3846153846152, 3338.4615384615386, 3444.6153846153848, 3553.846153846154, 3666.153846153846, 3781.5384615384614, 3900.0]], [[2500, 4000]])', b'Simulator...', b'[[2600.0, 2681.5384615384614, 2766.153846153846, 2853.846153846154, 2944.6153846153848, 3038.4615384615386, 3135.3846

concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 246, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 205, in _process_chunk
    return [fn(*args) for args in chunk]
  File "/usr/lib/python3.10/concurrent/futures/process.py", line 205, in <listcomp>
    return [fn(*args) for args in chunk]
  File "/home/amy/tmp/SimEx/notebooks/simulator.py", line 162, in sumo_simulator_novsl
    traci.start(sumoCmd)
  File "/home/amy/tmp/repos/SimEx/venv/lib/python3.10/site-packages/traci/main.py", line 145, in start
    sumoProcess = subprocess.Popen(cmd2, stdout=stdout)
  File "/usr/lib/python3.10/subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/usr/lib/python3.10/subprocess.py", line 1863, in _execute_child
    raise child_exception_type(errno_nu

 LINES: [b'Modifier...', b'[MODC]: (interval_min_tick, interval_max_tick):  (2500, 4000)', b'[MODC]: mod_ticks:  [2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900]', b'Temp x: [2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900]', b'[MODC]: mod_x:  [2600.0, 2681.5384615384614, 2766.153846153846, 2853.846153846154, 2944.6153846153848, 3038.4615384615386, 3135.3846153846152, 3235.3846153846152, 3338.4615384615386, 3444.6153846153848, 3553.846153846154, 3666.153846153846, 3781.5384615384614, 3900.0]', b'MAIN mod outcome ([[2600.0, 2681.5384615384614, 2766.153846153846, 2853.846153846154, 2944.6153846153848, 3038.4615384615386, 3135.3846153846152, 3235.3846153846152, 3338.4615384615386, 3444.6153846153848, 3553.846153846154, 3666.153846153846, 3781.5384615384614, 3900.0]], [[2500, 4000]])', b'Simulator...', b'[[2600.0, 2681.5384615384614, 2766.153846153846, 2853.846153846154, 2944.6153846153848, 3038.4615384615386, 3135.3846

IndexError: list index out of range

In [18]:
del mod_outcome

In [19]:
results

[[2810.0, 2870.0],
 [3320.0, 3360.0],
 [3400.0, 3520.0],
 [3530.0, 3750.0],
 [3760.0, 3810.0],
 [3890.0, 3900.0]]

In [20]:
mod_outcome = ModifierController.control(intervals_list=results, selected_modifier=components['modifierA'],
                                             do_plot=simexSettings['do_plot'])

Modifier...
[MODC]: (interval_min_tick, interval_max_tick):  (2810.0, 2870.0)
[MODC]: mod_ticks:  [2825, 2850]
[MODC]: mod_x:  [2825.0, 2850.0]
[MODC]: (interval_min_tick, interval_max_tick):  (3320.0, 3360.0)
[MODC]: mod_ticks:  [3325, 3350]
[MODC]: mod_x:  [3325.0, 3350.0]
[MODC]: (interval_min_tick, interval_max_tick):  (3400.0, 3520.0)
[MODC]: mod_ticks:  [3425, 3450, 3475, 3500]
[MODC]: mod_x:  [3425.0, 3449.8194945848377, 3474.8194945848377, 3500.0]
[MODC]: (interval_min_tick, interval_max_tick):  (3530.0, 3750.0)
[MODC]: mod_ticks:  [3550, 3575, 3600, 3625, 3650, 3675, 3700, 3725]
[MODC]: mod_x:  [3550.0, 3574.4845360824743, 3599.14089347079, 3623.9690721649486, 3648.9690721649486, 3674.14089347079, 3699.4845360824743, 3725.0]
[MODC]: (interval_min_tick, interval_max_tick):  (3760.0, 3810.0)
[MODC]: mod_ticks:  [3775, 3800]
[MODC]: mod_x:  [3775.0, 3800.0]
[MODC]: (interval_min_tick, interval_max_tick):  (3890.0, 3900.0)


In [21]:
mod_outcome

([[2825.0, 2850.0],
  [3325.0, 3350.0],
  [3425.0, 3449.8194945848377, 3474.8194945848377, 3500.0],
  [3550.0,
   3574.4845360824743,
   3599.14089347079,
   3623.9690721649486,
   3648.9690721649486,
   3674.14089347079,
   3699.4845360824743,
   3725.0],
  [3775.0, 3800.0]],
 [[2810.0, 2870.0],
  [3320.0, 3360.0],
  [3400.0, 3520.0],
  [3530.0, 3750.0],
  [3760.0, 3810.0],
  [3890.0, 3900.0]])